# Imports

In [ ]:
from artistplaylist import ArtistPlaylist
import event_scraper
import pprint

DAYS = 7
MAX_EVENTS = 150
MAX_SONG_COUNT = 10
EXCLUDE = ['Country', 'World', 'Latin', 'Hip-Hop', 'Gospel', 'Holiday', 'Christian', 'Children', 'Folk']

# Get Events

In [ ]:
events = event_scraper.get_events(DAYS, MAX_EVENTS)
print("Found {0} events".format(len(events)))
print('-' * 50)
for ev in events:
    print('{}  {:35}'.format(ev['date'], ev['artist']))

# ArtistPlaylist

## Print Functions

In [ ]:
def print_soung_counts(results_dict):
    print('-' * 50)
    for artist in list(results_dict.keys()):
        print('{:<3}songs|{}'.format(len(results_dict[artist]), artist))

def print_song_genres(song_list):
    print([song.get('genre', None) for song in song_list])
    
def print_song_list(song_list):
    s = '{:20}|{:35}|{:45}'
    print(s.format('Artist', 'Song Title', 'Genre'))
    for song in song_list:
        try:
            print(s.format(song['artist'], song['title'], str(song['genre'])))
        except KeyError:
            pass

## Instantiate

In [ ]:
import saving_artistplaylist
ap = saving_artistplaylist.SavingArtistPlaylist(saving_artistplaylist.SAVE_FILE)
#ap = ArtistPlaylist()

for ev in events:
    ap.search(ev['artist'])
    print('{:<3}songs|{}'.format(len(ap.results[ev['artist']]), ev['artist']))   

### First Look at Result Genres

In [ ]:
artists = list(ap.results.keys())
for a in artists:
    print('-' * 50)
    print(a)
    print_song_list(ap.results[a])

## Filter Functions

### Filters for entire results

In [ ]:
def filter_no_results(results_dict):
    return {artist: song_list for artist, song_list in results_dict.items() if not len(results_dict[artist]) == 0}

def filter_artist_mismatch(results_dict):
    return {artist: [song for song in song_list if song['artist'] == artist] for artist, song_list in results_dict.items()}

def filter_song_count(results_dict, min_count):
    return {artist: song_list for artist, song_list in results_dict.items() if len(song_list) >= min_count}

def filter_genre_blacklist(results_dict, blacklist):
    return {artist: songlist_filter_genre(song_list, blacklist) for artist, song_list in results_dict.items()}

### Filters for song lists

In [ ]:
def songlist_filter_genre(song_list, blacklist):
    return [song for song in filter_no_genre(song_list) if not any([ex in song.get('genre', ex) for ex in blacklist])]

def filter_no_genre(song_list):
    return [song for song in song_list if song.get('genre', None) is not None]

### Demonstrate filters

In [ ]:
print('-' * 50)
print("Starting:")
r = ap.results.copy()
print_soung_counts(r)

print('-' * 50)
print("Filtering for mismatched artists")
r = filter_artist_mismatch(r)
# print_soung_counts(r)

print('-' * 50)
print("Filtering for the genre blacklist")
r = filter_genre_blacklist(r, EXCLUDE)
# print_soung_counts(r)

print('-' * 50)
print("Filtering for the minimum song count")
r = filter_song_count(r, 10)
# print_soung_counts(r)

print('-' * 50)
print("Cleaning up the results")
r = filter_no_results(r)
print_soung_counts(r)

### Close look at results

In [ ]:
artists = list(r.keys())
for a in artists:
    print('-' * 50)
    print(a)
    print_song_list(r[a])

# Create list of song ids

In [ ]:
ap.save()
song_ids = [r[artist][0]['storeId'] for artist in list(r.keys())][:MAX_SONG_COUNT]
print(song_ids)

# Publishing Widget

In [ ]:
import ipywidgets as widgets
from IPython.display import display
button_pub = widgets.Button(description="Publish Playlist")
button_del = widgets.Button(description="Delete Playlist")
display(button_pub, button_del)

PUBLISH_PUBLICALLY = True

def pub_button_clicked(b):
    ap.create_playlist(song_ids, 'Austin Live Artists - V0.1', public=PUBLISH_PUBLICALLY)
    print("Published playlist")
    
def del_button_clicked(b):
    ap.delete_playlist()
    print("Deleted playlist")

button_pub.on_click(pub_button_clicked)
button_del.on_click(del_button_clicked)